# Data modeling plan 

LUT University \
BM20A6100 Advanced Data Analysis and Machine Learning - Blended teaching, Lpr 1.9.2025-12.12.2025 \
1st Period - Project work - Submission 2 - Data Modeling Plan

Elias Eskelinen, Vili Niemelä & Matti Aalto \
25.9.2025

---

## Data onboarding

In [3]:
import numpy as np

print("Hello, world!")

Hello, world!
